## 準備

In [1]:
# 4-3 Exercise1 cell1
from google.colab import drive
drive.mount("/content/drive")
%cd "/content/drive/MyDrive/WASEDADS/04/"

MessageError: Error: credential propagation was unsuccessful

In [ ]:
# 4-3 Exercise1 cell2
# ライブラリを色々インポート
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# 図に使う日本語フォントの設定
!pip install japanize-matplotlib | tail -n 1
import japanize_matplotlib

# 図の色の設定
col1 = "salmon"
col2 = "steelblue"

# データの用意
df = pd.read_excel("web_traffic.xlsx")
df = df.dropna()
df = df.reset_index(drop = True)

## 多項式回帰

In [ ]:
# 4-3 Exercise1 cell23
# 多項式の準備
k = 20 #最大次数
polynomial_features=PolynomialFeatures(degree = k, include_bias = False)
x_poly=polynomial_features.fit_transform(df_X.values.reshape(-1, 1))

# 多項式の正則化
sscaler=StandardScaler() # インスタンスの作成
sscaler.fit(x_poly) #x多項式の平均と標準偏差を計算
x_poly=sscaler.transform(x_poly) #xを標準化

###############
# 2. 多項式回帰を実行
###############
reg_poly = LinearRegression()
reg_poly.fit(x_poly, df_Y)


###############
# 3. アクセス数の予測
###############
x_pred = np.arange(0, 31, 0.1)
x_poly_pred = pf_x.fit_transform(x_pred.reshape(-1, 1))
x_poly_pred = sscaler.transform(x_poly_pred)
y_pred = reg_poly.predict(x_poly_pred)

###############
# 4. 結果の図示
###############
plt.scatter(df_X, df_Y, s = 50, c = col2, edgecolors = "white")
plt.plot(x_pred, y_pred, color = col1, linewidth = 3)
plt.plot([31, 31], [0, 7], c = "k", linestyle = "dashed", linewidth = 1)
plt.xlabel("時間（日）", fontsize = 15)
plt.ylabel("ページビュー (×1000回)", fontsize = 15)
plt.xlim([0, 31])
plt.ylim([0, 7])
plt.title("多項式回帰 (k=" + str(k) + ")", fontsize = 15)

## Lasso回帰

In [ ]:
# 4-3 Exercise1 cell18
######################################################
# Lasso回帰を実行する
######################################################
#sklearn.linear_modelからLassoをインポート
from sklearn.linear_model import Lasso

# αを定義
Alp = 0.1

# Lasso回帰の分析器の設定
reg_lasso = Lasso(alpha = Alp)

# 関係式のパラメータの推定
reg_lasso.fit(x_poly, df_Y)

# 予測
y_pred_lasso = reg_lasso.predict(x_poly_pred)
######################################################

######################################################
# 結果の図示
######################################################
plt.scatter(df_X, df_Y, s = 50, c = col2, edgecolors = "white", alpha = 0.5)
plt.plot(x_pred, y_pred_lasso, color = col1, linewidth = 3)
plt.axvline(x = 23, color = "k", ls = ":")
plt.axvline(x = 27, color = "k", ls = ":")
plt.xlabel("時間（日）", fontsize = 15)
plt.ylabel("ページビュー (×1000回)", fontsize = 15)
plt.title("Lasso回帰 (k=" + str(k) + ", alpha=" + str(Alp) + ")", fontsize = 15);
######################################################

## Ridge回帰

In [ ]:
# 4-3 Exercise1 cell19
######################################################
# Ridge回帰を実行する
######################################################
#sklearn.linear_modelからRidgeをインポート
from sklearn.linear_model import Ridge

# αを定義
Alp = 0.1

# Ridge回帰の分析器の設定
reg_ridge = Ridge(alpha = Alp)

# 関係式のパラメータの推定
reg_ridge.fit(x_poly, df_Y)

# 最適なモデルを使って予測
y_pred_redge = reg_ridge.predict(x_poly_pred)

######################################################

######################################################
#結果の図示
######################################################
plt.scatter(df_X, df_Y, s = 50, c = col2, edgecolors = "white", alpha = 0.5)
plt.plot(x_pred, y_pred_redge, color = col1, linewidth = 3)
plt.xlabel("時間（日）", fontsize = 15)
plt.ylabel("ページビュー (×1000回)", fontsize = 15)
plt.axvline(x = 23, color = "k", ls = ":")
plt.axvline(x = 27, color = "k", ls = ":")
plt.title("Ridge回帰 (k=" + str(k) + ", alpha=" + str(Alp) + ")", fontsize = 15);
######################################################

# αの推定

## Lasso/Ridge

In [ ]:
# 4-3 Exercise2 cell14
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
###########################
# 正則化パラメータαの定義
###########################
logAlphas = np.arange(-1, 5, 1.0)
Alphas = 10**logAlphas

###########################
#クロスバリデーション
###########################
#多項式回帰式の最大次数Kを定義
k = 20
#クロスバリデーションの分割数Kを定義
k_fold = 10

mse = np.zeros(len(Alphas)) # 各αに対する平均二乗誤差を格納する配列を配列を準備

for idx in range(len(Alphas)): # for文を使って各αに対する平均二乗誤差を計算
  Alp = Alphas[idx]
  reg_ridge = Ridge(alpha = Alp, max_iter = 100000)
  kf = KFold(n_splits = k_fold, shuffle = True, random_state = 1)
  scores = cross_validate(reg_ridge, x_poly, df_Y, cv = kf, scoring = "neg_mean_squared_error")
  mse[idx]=np.mean(-1*scores["test_score"])

###########################
#　最適な（平均二乗誤差を最小の）αを選択
###########################
idx_opt = np.argmin(mse)
Alpha_opt = Alphas[idx_opt]

###########################
#　クロスバリデーションの結果の図示
###########################
plt.plot(Alphas, mse) # 平均二乗誤差　vs α
plt.scatter(Alpha_opt, mse[idx_opt]) # 最適なαとその平均二乗誤差を表す点をプロット
plt.xscale("log")
plt.xlabel("正則化パラメータ α", fontsize = 15)
plt.ylabel("平均二乗誤差", fontsize = 15)
plt.show()
print("")

###########################
#　最適なαを用いた回帰曲線
###########################
reg_ridge = Ridge(alpha = Alpha_opt, max_iter = 100000)
reg_ridge.fit(x_poly, df_Y)
y_pred_ridge = reg_ridge.predict(x_poly_pred)


###########################
# 予測結果の図示
###########################
plt.scatter(df_X, df_Y, s = 50, c = col2, edgecolors = "white", alpha = 0.5) # 元データ
plt.plot(x_pred, y_pred_ridge, color = col1, linewidth = 3) # 回帰曲線
plt.xlabel("時間（日）", fontsize = 15)
plt.ylabel("ページビュー (×1000回)", fontsize = 15)
plt.title("Lasso回帰 (k=" + str(k) + ", alpha=" + str(Alpha_opt) + ")", fontsize = 15);
plt.show()
print("")

###########################
# 回帰係数を図示
###########################
plt.figure(figsize=(8, 4))
plt.plot([0, 21], [0, 0], c = "k", linestyle = "solid", linewidth = 2)
plt.scatter(np.arange(len(reg_ridge.coef_)) + 1, reg_ridge.coef_ , s = 300, c = col1, edgecolors = "white", marker = "d")
plt.ylabel("回帰係数 β", fontsize = 15)
plt.xlabel("次数 i", fontsize = 15)
plt.xticks(np.arange(0, 21, 5))
plt.yticks(np.arange(0.0, 0.45, 0.1))
plt.ylim([-0.1, 0.5])
plt.grid()